## Deliverable 2. Create a Customer Travel Destinations Map.

In [95]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

### 1. Import the WeatherPy_database.csv file. 

In [9]:
# Read the CSV file and store it in dataframe
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,61.88,85,40,1.99,scattered clouds
1,1,San Quintin,MX,30.4833,-115.9500,58.35,87,0,8.12,clear sky
2,2,Lebu,CL,-37.6167,-73.6500,55.24,85,1,1.88,clear sky
3,3,Iqaluit,CA,63.7506,-68.5145,10.13,72,90,13.00,overcast clouds
4,4,Metro,ID,-5.1131,105.3067,85.23,68,100,8.81,moderate rain


### 2. Prompt the user to enter minimum and maximum temperature criteria 

In [11]:
# Enter the minimum and maximum temperature.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


### 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

In [12]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Metro,ID,-5.1131,105.3067,85.23,68,100,8.81,moderate rain
6,6,Mahebourg,MU,-20.4081,57.7000,82.69,65,40,13.80,scattered clouds
17,17,Atuona,PF,-9.8000,-139.0333,78.51,78,7,18.81,clear sky
20,20,Busselton,AU,-33.6500,115.3333,78.15,37,22,5.99,few clouds
22,22,Hobart,AU,-42.8794,147.3294,76.26,47,75,8.01,broken clouds
24,24,Conde,BR,-7.2597,-34.9075,78.26,89,20,6.91,few clouds
27,27,Port Blair,IN,11.6667,92.7500,81.97,58,75,12.97,broken clouds
29,29,Luanda,AO,-8.8368,13.2343,77.72,83,100,5.75,overcast clouds
34,34,Cabo San Lucas,MX,22.8909,-109.9124,75.51,72,100,5.41,overcast clouds
36,36,Hilo,US,19.7297,-155.0900,79.29,89,40,10.36,moderate rain


### 4a. Determine if there are any empty rows.

In [17]:
# Check for empty rows
preferred_cities_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Weather Description    186
dtype: int64

### 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [105]:
# Drop if there are any empty cells
clean_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Weather Description    186
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Metro,ID,85.23,moderate rain,-5.1131,105.3067,
6,Mahebourg,MU,82.69,scattered clouds,-20.4081,57.7000,
17,Atuona,PF,78.51,clear sky,-9.8000,-139.0333,
20,Busselton,AU,78.15,few clouds,-33.6500,115.3333,
22,Hobart,AU,76.26,broken clouds,-42.8794,147.3294,
24,Conde,BR,78.26,few clouds,-7.2597,-34.9075,
27,Port Blair,IN,81.97,broken clouds,11.6667,92.7500,
29,Luanda,AO,77.72,overcast clouds,-8.8368,13.2343,
34,Cabo San Lucas,MX,75.51,overcast clouds,22.8909,-109.9124,
36,Hilo,US,79.29,moderate rain,19.7297,-155.0900,


In [75]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # 6e. Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [113]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [115]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [106]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [107]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))